# 1. Cleaning code of 'fatal-police-shootings-data'

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import os

drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/WQD7001_Project'
raw_data_path = os.path.join(base_dir, 'data', 'raw', 'fatal-police-shootings-data.csv')
processed_data_path = os.path.join(base_dir, 'data', 'processed')

os.makedirs(processed_data_path, exist_ok=True)

# Define non-core qualitative columns for uniform 'Unknown/Missing' filling
non_core_qualitative_cols = ['county', 'latitude', 'longitude', 'location_precision', 'race_source', 'agency_ids']

print(f"Loading data from: {raw_data_path}")
try:
    df = pd.read_csv(raw_data_path, encoding='latin-1')
    print("Data loaded successfully with encoding 'latin-1'.")
except UnicodeDecodeError:
    df = pd.read_csv(raw_data_path)
    print("Data loaded with default encoding (UTF-8).")
except FileNotFoundError:
    print(f"Error: File not found at  {raw_data_path}")

#Ensure there are no spaces in column names and obtain the raw data volume
df.columns = df.columns.str.strip()
print(f"Raw data volume：{len(df)} records")
initial_rows = len(df)

# 1. Format Standardized
# 1.1 Filter data from 2018 to 2024（inclusive）
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df = df[(df['year'] >= 2020) & (df['year'] <= 2024)].copy()
print(f"1.1 Data Filtering: records from 2020 through 2024")

# 1.2 Create new row id starting from 1
df = df.reset_index(drop=True)
df['row_id'] = df.index + 1
print(f"1.2 A total of {len(df)} records remain after filtering and re-indexing")

# 2. Duplicate Records Handled
count_before_dedupe = len(df)
df.drop_duplicates(subset=['name', 'date', 'city', 'state'], keep='first', inplace=True)
records_removed_by_dedupe = count_before_dedupe - len(df)
print(f"2.1 Duplicate Records Handled: Removed {records_removed_by_dedupe} records")

# 3. Missing Values Handled
# 3.1 Drop missing core identifiers, situational indicators
df.dropna(subset=['name', 'date', 'city', 'state'], inplace=True)

print("Starting Age Imputation...")
df['age'] = df['age'].fillna(df.groupby(['race', 'gender'])['age'].transform('median'))
df['age'] = df['age'].fillna(df.groupby(['race'])['age'].transform('median'))
df['age'] = df['age'].fillna(df['age'].median())
print("3.x Age imputed using hierarchical median strategy (Race/Gender -> Race -> Global)")

# 3.2 fill situational indicators with unknown (Modified based on teammate request)
# Logic update: fillna with 'unknown', replace 'undetermined' with 'unknown'
df['threat_type'] = df['threat_type'].fillna('unknown').replace('undetermined', 'unknown')
df['flee_status'] = df['flee_status'].fillna('unknown').replace('undetermined', 'unknown')
df['armed_with'] = df['armed_with'].fillna('unknown').replace('undetermined', 'unknown')
# Logic update: gender 'non-binary' to 'unknown'
df['gender'] = df['gender'].fillna('unknown').replace('non-binary', 'unknown')
print("3.2 Situational indicators (threat/flee/armed) and gender consolidated to unknown")

# 3.3 Non-Core Qualitative Variables: Indicator Category Imputation
for col in non_core_qualitative_cols:
    df[col] = df[col].fillna('Unknown/Missing')
print(f"3.3 Non-core fields were unified to 'Unknown/Missing'")

# 3.4 Repair and standardize irregular text values for 'city' and 'county'
df['city'] = df['city'].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df['county'] = df['county'].astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df['city'] = df['city'].str.replace(r'asley', 'easley', regex=True, case=False)
print(f"3.4 Repaired and standardized irregular text values for 'city' and 'county'")

# 3.5 Standardize geographical names for consistency
df['city'] = df['city'].str.replace(r'[.,]', '', regex=True)
df['county'] = df['county'].str.replace(r'[.,]', '', regex=True)
df['city'] = df['city'].str.lower().str.strip()
df['county'] = df['county'].str.lower().str.strip()
df['state'] = df['state'].str.upper().str.strip()
print("3.5 Geographical names normalized, punctuation removed, and casing standardized")

# 4. Feature Engineering
# 4.1 Decomposing the 'armed_with' column
df['armed_with'] = df['armed_with'].astype(str)
weapon_keywords_main = ['gun', 'knife', 'blunt_object', 'vehicle', 'replica']
for keyword in weapon_keywords_main:
    df[f'has_{keyword}'] = df['armed_with'].str.contains(keyword, case=False, na=False).astype(int)

df['has_unarmed'] = df['armed_with'].str.contains('unarmed', case=False, na=False).astype(int)

# Logic update: has_other should NOT include 'unknown' or 'undetermined'
other_keywords_for_consolidation = ['other'] # removed unknown and undetermined
other_pattern = '|'.join(other_keywords_for_consolidation)
df['has_other'] = df['armed_with'].str.contains(other_pattern, case=False, na=False).astype(int)
print(f"4.1 Decomposing the 'armed_with' column: 7 binary weapon features created.")

# 4.2 Race Feature Engineering
df['race_std'] = df['race'].astype(str).str.upper().str.strip()
is_missing_race = (df['race_std'] == 'NAN') | (df['race'].isna())

df['share_white'] = df['race_std'].str.contains('W', case=False, na=False)
df['share_black'] = df['race_std'].str.contains('B', case=False, na=False)
df['share_native_american'] = df['race_std'].str.contains('N', case=False, na=False)
df['share_asian'] = df['race_std'].str.contains('A', case=False, na=False)
df['share_hispanic'] = df['race_std'].str.contains('H', case=False, na=False)
df['share_two_or_more'] = df['race_std'].str.contains(r'[;,]', regex=True, na=False)

df['share_missing'] = is_missing_race.astype(int)

race_cols = ['share_white', 'share_black', 'share_native_american',
             'share_asian', 'share_hispanic', 'share_two_or_more']

for col in race_cols:
    df[col] = df[col].astype(int)
    df.loc[is_missing_race, col] = 0

print(f"4.2 Race Feature Engineering: Six overlapping binary share features created")

#5. Outlier Treatment Handled
df['armed_with'] = df['armed_with'].astype(str)
# Added 'unknown' to conflict list just in case, though logically it shouldn't conflict with unarmed if handled correctly
weapon_keywords_for_conflict = ['gun', 'knife', 'blunt_object', 'vehicle', 'other', 'replica', 'unknown', 'undetermined']
conflict_pattern = r'|'.join(weapon_keywords_for_conflict)
rows_to_drop_conflict = df[
    df['armed_with'].str.contains(r'unarmed', case=False, na=False) &
    df['armed_with'].str.contains(conflict_pattern, case=False, na=False)].index
df.drop(rows_to_drop_conflict, inplace=True)
print(f"5.1 Outlier Treatment Handled: Removed {len(rows_to_drop_conflict)} records with conflicting 'unarmed' status")

upper_bound = df['age'].quantile(0.99)
df['age'] = df['age'].clip(upper=upper_bound)
print(f"5.2 Outlier Treatment Handled: 'age' capped at 99th percentile ({upper_bound:.1f})")


# Calculate the number of records after cleaning
cleaned_rows = len(df)
print(f"Cleaned data volume: {cleaned_rows} records")

# Save the cleaned dataset
# [修改说明]: 保存到 Google Drive 的 processed 文件夹
output_main_path = os.path.join(processed_data_path, 'cleaned_fatal_shootings_data_2020_2024-newmain.csv')
df.to_csv(output_main_path, index=False)
print(f"Cleaned data saved to '{output_main_path}'")

#6. Final Feature Selection
columns_to_drop = ['id', 'date','county', 'name', 'race_source', 'location_precision', 'agency_ids']
df.drop(columns=columns_to_drop, inplace=True)
print("6.1 Final Feature Selection: Dropped low-value/redundant columns")

# Calculate the number of records after selected
final_rows = len(df)
print(f"Final data volume: {final_rows} records.")


output_selected_path = os.path.join(processed_data_path, 'cleaned_selected_features_2020_2024.csv')
df.to_csv(output_selected_path, index=False)
print(f"Final data saved to '{output_selected_path}'")

Mounted at /content/drive
Loading data from: /content/drive/MyDrive/WQD7001_Project/data/raw/fatal-police-shootings-data.csv
Data loaded successfully with encoding 'latin-1'.
Raw data volume：10430 records
1.1 Data Filtering: records from 2020 through 2024
1.2 A total of 5507 records remain after filtering and re-indexing
2.1 Duplicate Records Handled: Removed 0 records
Starting Age Imputation...
3.x Age imputed using hierarchical median strategy (Race/Gender -> Race -> Global)
3.2 Situational indicators (threat/flee/armed) and gender consolidated to unknown
3.3 Non-core fields were unified to 'Unknown/Missing'
3.4 Repaired and standardized irregular text values for 'city' and 'county'
3.5 Geographical names normalized, punctuation removed, and casing standardized
4.1 Decomposing the 'armed_with' column: 7 binary weapon features created.
4.2 Race Feature Engineering: Six overlapping binary share features created
5.1 Outlier Treatment Handled: Removed 0 records with conflicting 'unarmed'

# 2. Cleaning code of 'socioeconomic table' from 2020-2024

In [4]:
from google.colab import drive
import pandas as pd
import numpy as np
import os

drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/WQD7001_Project'

raw_data_dir = os.path.join(base_dir, 'data', 'raw')
processed_data_dir = os.path.join(base_dir, 'data', 'processed')

os.makedirs(processed_data_dir, exist_ok=True)

# Define Census codes to target metric names
FINAL_RENAME_MAP = {
    'DP03_0128PE': 'poverty_rate',
    'DP02_0062PE': 'high_school_grad_rate',
    'DP03_0062E': 'median_household_income',
    'DP05_0037PE': 'share_white',
    'DP05_0038PE': 'share_black','DP05_0045PE': 'share_black',
    'DP05_0039PE': 'share_native_american','DP05_0053PE': 'share_native_american',
    'DP05_0044PE': 'share_asian', 'DP05_0047PE': 'share_asian', 'DP05_0061PE': 'share_asian',
    'DP05_0071PE': 'share_hispanic', 'DP05_0074PE': 'share_hispanic', 'DP05_0076PE': 'share_hispanic', 'DP05_0090PE': 'share_hispanic',
    'DP05_0035PE': 'share_two_or_more', 'DP05_0033E': 'total_population',
    'state': 'state_code'}

# Define auxiliary files and their corresponding years
file_maps = [('socioeconomic_2020.csv', 2020), ('socioeconomic_2021.csv', 2021), ('socioeconomic_2022.csv', 2022), ('socioeconomic_2023.csv', 2023), ('socioeconomic_2024.csv', 2024)]

all_auxiliary_data = []

# State Full Name to Abbreviation Map
state_abbr_map = {
    'ALABAMA': 'AL', 'ALASKA': 'AK', 'ARIZONA': 'AZ', 'ARKANSAS': 'AR', 'CALIFORNIA': 'CA',
    'COLORADO': 'CO', 'CONNECTICUT': 'CT', 'DELAWARE': 'DE', 'DISTRICT OF COLUMBIA': 'DC',
    'FLORIDA': 'FL', 'GEORGIA': 'GA', 'HAWAII': 'HI', 'IDAHO': 'ID', 'ILLINOIS': 'IL',
    'INDIANA': 'IN', 'IOWA': 'IA', 'KANSAS': 'KS', 'KENTUCKY': 'KY', 'LOUISIANA': 'LA',
    'MAINE': 'ME', 'MARYLAND': 'MD', 'MASSACHUSETTS': 'MA', 'MICHIGAN': 'MI', 'MINNESOTA': 'MN',
    'MISSISSIPPI': 'MS', 'MISSOURI': 'MO', 'MONTANA': 'MT', 'NEBRASKA': 'NE', 'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH', 'NEW JERSEY': 'NJ', 'NEW MEXICO': 'NM', 'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC', 'NORTH DAKOTA': 'ND', 'OHIO': 'OH', 'OKLAHOMA': 'OK', 'OREGON': 'OR',
    'PENNSYLVANIA': 'PA', 'RHODE ISLAND': 'RI', 'SOUTH CAROLINA': 'SC', 'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN', 'TEXAS': 'TX', 'UTAH': 'UT', 'VERMONT': 'VT', 'VIRGINIA': 'VA',
    'WASHINGTON': 'WA', 'WEST VIRGINIA': 'WV', 'WISCONSIN': 'WI', 'WYOMING': 'WY',
    'PUERTO RICO': 'PR'
}

print("Starting to load auxiliary files...")

for file_name, year in file_maps:
    file_path = os.path.join(raw_data_dir, file_name)

    try:
# 1. Load file with robust encoding
        df_temp = pd.read_csv(file_path, encoding='latin-1')
        df_temp.columns = df_temp.columns.str.strip()

# 2. Select columns and rename
        available_cols = ['NAME'] + [col for col in df_temp.columns if col in FINAL_RENAME_MAP or col == 'state']
        df_temp = df_temp[available_cols].copy()
        df_temp = df_temp.rename(columns=FINAL_RENAME_MAP)
        df_temp['year'] = year
        all_auxiliary_data.append(df_temp)
        print(f"Loaded {file_name} successfully.")
    except FileNotFoundError:
        print(f"❌ Error: File not found at {file_path}")

# 3. Create the Auxiliary Master Table (Vertical Concatenation)
if all_auxiliary_data:
    df_auxiliary_master = pd.concat(all_auxiliary_data, ignore_index=True)
    print(f"Auxiliary Master Table created with {len(df_auxiliary_master)} total records.")

    raw_output_path = os.path.join(processed_data_dir, 'auxiliary_socioeconomic_master_table_raw.csv')
    df_auxiliary_master.to_csv(raw_output_path, index=False)
    print(f"\nAuxiliary Master Table successfully saved to '{raw_output_path}'")

    df = pd.read_csv(raw_output_path)

    # 4. Create a new column containing the abbreviations for the names of states.
    df['state_abbr'] = df['NAME'].astype(str).str.upper().str.strip().map(state_abbr_map)

    cols = df.columns.tolist()
    if 'NAME' in cols and 'state_abbr' in cols:
        name_idx = cols.index('NAME')
        cols.insert(name_idx + 1, cols.pop(cols.index('state_abbr')))
        df = df[cols]

    # 5. Use nan to fill missing value
    target_cols = [
        'poverty_rate',
        'high_school_grad_rate',
        'median_household_income',
        'share_white',
        'share_black',
        'share_native_american',
        'share_asian',
        'share_hispanic',
        'share_two_or_more',
        'total_population']

    missing_values_to_replace = ['', ' ', '#N/A', 'NULL', 'null', 'None', '-']

    for col in target_cols:
        if col in df.columns:
            df[col] = df[col].replace(missing_values_to_replace, 'nan')
            df[col] = df[col].fillna('nan')

    # 6. Save final processed file
    final_output_path = os.path.join(processed_data_dir, 'auxiliary_socioeconomic_processed.csv')
    df.to_csv(final_output_path, index=False)
    print(f"Successfully processed and saved to {final_output_path}")

else:
    print("No data loaded. Please check your raw data files.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting to load auxiliary files...
Loaded socioeconomic_2020.csv successfully.
Loaded socioeconomic_2021.csv successfully.
Loaded socioeconomic_2022.csv successfully.
Loaded socioeconomic_2023.csv successfully.
Loaded socioeconomic_2024.csv successfully.
Auxiliary Master Table created with 260 total records.

Auxiliary Master Table successfully saved to '/content/drive/MyDrive/WQD7001_Project/data/processed/auxiliary_socioeconomic_master_table_raw.csv'
Successfully processed and saved to /content/drive/MyDrive/WQD7001_Project/data/processed/auxiliary_socioeconomic_processed.csv
